In [ ]:
import torch

In [ ]:
from transformers import BertTokenizer

In [ ]:
import json

# Load JSON data
with open('C:/Users/ObelWork-1/Downloads/data_for_vectordb.json', 'r') as file:
    data = json.load(file)

# Extract descriptions and CVE IDs
descriptions = [item['description'] for item in data]
cve_ids = [item['id'] for item in data]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
encoded_inputs = tokenizer(descriptions, padding=True, truncation=True, return_tensors='pt')

In [ ]:
token_ids = encoded_inputs['input_ids']
attention_masks = encoded_inputs['attention_mask']

In [ ]:
from transformers import AutoModel
import torch

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)

In [ ]:
def embed_tokens(input_ids, attention_mask):
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    # Use the [CLS] token representation for sentence-level embedding
    embeddings = outputs.last_hidden_state[:, 0, :]  # Shape: [batch_size, hidden_dim]
    return embeddings

In [ ]:
batch_size = 1024  # Adjust if necessary
all_embeddings = []

for i in range(0, len(token_ids), batch_size):
    batch_input_ids = torch.tensor(token_ids[i:i+batch_size])  # [batch_size, seq_length]
    batch_attention_mask = torch.tensor(attention_masks[i:i+batch_size])  # [batch_size, seq_length]

    batch_embeddings = embed_tokens(batch_input_ids, batch_attention_mask)
    all_embeddings.append(batch_embeddings)

# Concatenate all embeddings into one tensor
all_embeddings = torch.cat(all_embeddings, dim=0)


C:\Users\ObelWork-1\AppData\Local\Temp\ipykernel_15984\3592697210.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_input_ids = torch.tensor(token_ids[i:i+batch_size])  # [batch_size, seq_length]
C:\Users\ObelWork-1\AppData\Local\Temp\ipykernel_15984\3592697210.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_attention_mask = torch.tensor(attention_masks[i:i+batch_size])  # [batch_size, seq_length]


In [ ]:
torch.save(all_embeddings, "embeddings.pt")
